In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

def split_dataset(input_csv, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, random_state=42):
    """
    Dzieli dane na zbiory treningowy, walidacyjny i testowy z zachowaniem proporcji klas i grupowania po ziarnach.

    Args:
        input_csv (str): Ścieżka do pliku CSV z danymi.
        train_ratio (float): Proporcja danych treningowych.
        val_ratio (float): Proporcja danych walidacyjnych.
        test_ratio (float): Proporcja danych testowych.
        random_state (int): Losowy seed dla powtarzalności.

    Returns:
        train_df, val_df, test_df: Dane podzielone na zbiory.
    """
    # Wczytaj dane
    df = pd.read_csv(input_csv)

    # Grupowanie ziaren na podstawie unikalnego `id`
    grouped = df.groupby('id').first()  # Wybieramy reprezentatywny wiersz dla każdego ziarna
    ids = grouped.index
    classes = grouped['class']

    # Inicjalizacja StratifiedShuffleSplit
    sss = StratifiedShuffleSplit(n_splits=1, test_size=(val_ratio + test_ratio), random_state=random_state)

    # Podział na zbiór treningowy i tymczasowy (walidacja + test)
    for train_idx, temp_idx in sss.split(ids, classes):
        train_ids = ids[train_idx]
        temp_ids = ids[temp_idx]

    # Podział tymczasowego zbioru na walidację i test
    sss_temp = StratifiedShuffleSplit(n_splits=1, test_size=test_ratio / (val_ratio + test_ratio), random_state=random_state)
    for val_idx, test_idx in sss_temp.split(temp_ids, classes[temp_idx]):
        val_ids = temp_ids[val_idx]
        test_ids = temp_ids[test_idx]

    # Tworzenie zbiorów na podstawie podzielonych `id`
    train_df = df[df['id'].isin(train_ids)]
    val_df = df[df['id'].isin(val_ids)]
    test_df = df[df['id'].isin(test_ids)]

    return train_df, val_df, test_df

# Przykład użycia
input_csv = "CSV/dataset/dataset.csv"  # Plik wejściowy CSV
train_df, val_df, test_df = split_dataset(input_csv)

# Zapisz zbiory do plików CSV
train_df.to_csv("CSV/dataset/train.csv", index=False)
val_df.to_csv("CSV/dataset/val.csv", index=False)
test_df.to_csv("CSV/dataset/test.csv", index=False)

# Wyświetl liczność zbiorów
print(f"Liczba ziaren w zbiorze treningowym: {train_df['id'].nunique()} (obrazy: {len(train_df)})")
print(f"Liczba ziaren w zbiorze walidacyjnym: {val_df['id'].nunique()} (obrazy: {len(val_df)})")
print(f"Liczba ziaren w zbiorze testowym: {test_df['id'].nunique()} (obrazy: {len(test_df)})")


Liczba ziaren w zbiorze treningowym: 69506 (obrazy: 208509)
Liczba ziaren w zbiorze walidacyjnym: 14894 (obrazy: 44680)
Liczba ziaren w zbiorze testowym: 14895 (obrazy: 44682)
